In [47]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizerFast
from collections import Counter
import re
import csv

import sklearn
from torch import nn
import numpy as np

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import pandas as pd
from sklearn.metrics import multilabel_confusion_matrix

In [48]:
NUM_CLASSES = 6
NUM_SENTIMENT = 3
NUM_HATE = 2

BATCH_SIZE=8
EPOCHS=1

In [49]:
trainDatasetPath = './train.csv'

class ToxicCommentsRobertaDataset(Dataset):
    def __init__(self, path, columnName, tokenizer, maxLength=512):
        """
        Args:
            path (string): Path to the csv file with annotations.
            columnName (string): Name of the column containing the comments.
            tokenizer: Tokenizer to use for tokenizing the comments.
            maxLength (int): Maximum length of the tokenized comments.
        """
        self.texts = []
        self.labels_list = []
        self.tokenizer = tokenizer
        self.maxLength = maxLength
        with open(path, encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader)
            textColIndex = header.index(columnName)
            labelColIndices = [
                header.index(label)
                for label in [
                    "toxic",
                    "severe_toxic",
                    "obscene",
                    "threat",
                    "insult",
                    "identity_hate",
                ]
            ]
            printed = False
            for rowIndex, row in enumerate(reader):
                text = row[textColIndex]
                # Store all label columns independently
                labels = [int(row[labelColIndex]) for labelColIndex in labelColIndices] # Changed line
                # Removing aggregation logic
                # if not printed:
                #     print(labels)
                #     printed = True


                self.texts.append(text)
                self.labels_list.append(labels)
        self.labels = torch.tensor(self.labels_list, dtype=torch.float32) # Changed line
        # Calculate weights for each label independently
        pos_labels = self.labels.sum(dim=0) # Changed line
        neg_labels = self.labels.shape[0] - pos_labels # Changed line
        self.weights = neg_labels/pos_labels # Changed line

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index: int):
        text = self.texts[index]
        labels = self.labels[index]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.maxLength,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": labels,
        }

In [50]:
class distilRB_base(nn.Module):
    def __init__(self, multilabel: bool=False):
        super(distilRB_base, self).__init__()
        self.base_model = AutoModel.from_pretrained("distilbert/distilroberta-base")
        self.activation = nn.Linear(self.base_model.config.hidden_size, NUM_CLASSES if multilabel else 1)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
        inter = self.base_model(input_ids, attention_mask)
        pooled = inter.pooler_output
        y_hat = self.activation(pooled)
        return y_hat

class distilRB_hate(nn.Module):
    def __init__(self, multilabel: bool=False):
        super(distilRB_hate, self).__init__()
        self.base_model = AutoModel.from_pretrained("distilbert/distilroberta-base")
        self.hate_model = AutoModelForSequenceClassification.from_pretrained("tomh/toxigen_roberta")
        for param in self.hate_model.parameters():
            param.requires_grad = False
        #self.hate_model output for forward is a logit of size 2
        self.activation = nn.Linear(self.base_model.config.hidden_size + NUM_HATE, NUM_CLASSES if multilabel else 1)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
        inter = self.base_model(input_ids, attention_mask)
        pooled = inter.pooler_output
        hate_signal = self.hate_model(input_ids, attention_mask).logits
        combined = torch.cat([pooled, hate_signal], dim=1)

        y_hat = self.activation(combined)
        return y_hat


class distilRB_sem(nn.Module):
    def __init__(self, multilabel: bool=False):
        super(distilRB_sem, self).__init__()
        self.base_model = AutoModel.from_pretrained("distilbert/distilroberta-base")
        self.sent_anal = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
        for param in self.sent_anal.parameters():
            param.requires_grad = False
        #self.sentiment output for forward is a logit of size 3
        self.activation = nn.Linear(self.base_model.config.hidden_size + NUM_SENTIMENT, NUM_CLASSES if multilabel else 1)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
        inter = self.base_model(input_ids, attention_mask)
        pooled = inter.pooler_output
        sentiment = self.sent_anal(input_ids, attention_mask)
        combined = torch.cat([pooled, sentiment], dim=1)
        y_hat = self.activation(combined)
        return y_hat

class distilRB_combine(nn.Module):
    def __init__(self, multilabel: bool=False):
        super(distilRB_combine, self).__init__()
        self.base_model = AutoModel.from_pretrained("distilbert/distilroberta-base")
        self.sent_anal = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
        self.hate_model = AutoModelForSequenceClassification.from_pretrained("tomh/toxigen_roberta")
        for param in self.sent_anal.parameters():
            param.requires_grad = False

        for param in self.hate_model.parameters():
            param.requires_grad = False
        #self.sentiment output for forward is a logit of size 3
        self.activation = nn.Linear(
            self.base_model.config.hidden_size + NUM_SENTIMENT + NUM_HATE, NUM_CLASSES if multilabel else 1
        )

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
        inter = self.base_model(input_ids, attention_mask)
        pooled = inter.pooler_output
        sentiment = self.sent_anal(input_ids, attention_mask)
        hate = self.hate_model(input_ids, attention_mask)
        combined = torch.cat([pooled, sentiment, hate], dim=1)
        y_hat = self.activation(combined)
        return y_hat


In [51]:
def trim(batch: dict[str, torch.tensor]):
    masks = batch['attention_mask']
    max_len = torch.max(torch.sum(masks, dim=1))

    if batch['labels'].shape == batch['input_ids'].shape:
        batch['labels'] = batch['labels'][:, :max_len]

    batch['input_ids'] = batch['input_ids'][:, :max_len]
    batch['attention_mask'] = batch['attention_mask'][:, :max_len]

    return batch

def train_model(model, dataset: Dataset, lr=1e-5, weight_decay=1e-3):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(lr=lr, weight_decay=weight_decay, params=model.parameters())


    train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    loss_per_epoch = []

    criterion = torch.nn.BCEWithLogitsLoss()

    model.train()
    for epoch in range(EPOCHS):
        losses = []
        for batch in train_dataloader:
            batch = trim(batch)

            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            # Cast labels to float16 before loss calculation
            labels = batch["labels"].to(device) # Changed line
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(**inputs).squeeze()


            loss = criterion(outputs, labels)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_per_epoch.append(losses)

    return loss_per_epoch

In [61]:
def evaluate_model(model, dataset: ToxicCommentsRobertaEvalDataset, device='cuda'):
    model.to(device)
    model.eval()

    eval_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)  # Create DataLoader

    preds = []
    true_labels = []

    with torch.no_grad():
        for batch in eval_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)

            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(**inputs).squeeze()

            probs = torch.sigmoid(outputs)
            pred = (probs > 0.5).int().cpu().numpy()

            preds.extend(pred)  # Extend the list of predictions
            true_labels.extend(labels.cpu().numpy())  # Extend the list of true labels

    # Convert preds to a NumPy array with the correct shape
    preds = np.array(preds)
    true_labels = np.array(true_labels) # Ensure true_labels is an integer array

    print(preds.shape)
    print(true_labels.shape)

    # Calculate overall confusion matrix
    cm = multilabel_confusion_matrix(true_labels, preds)

    f1 = sklearn.metrics.f1_score(true_labels, preds, average='macro')
    accuracy = sklearn.metrics.accuracy_score(true_labels, preds)
    precision = sklearn.metrics.precision_score(true_labels, preds, average='macro')
    recall = sklearn.metrics.recall_score(true_labels, preds, average='macro')

    metrics = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1)
    }

    return metrics, preds, true_labels

In [53]:
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_cached()/1024**2)


if __name__ == '__main__' :
    memory_stats()
    torch.cuda.empty_cache()
    memory_stats()
    tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
    toxicDataset = ToxicCommentsRobertaDataset(trainDatasetPath, 'comment_text', tokenizer=tokenizer)

    # if len(toxicDataset) > 0:
    #     print(f"\nLength of dataset: {len(toxicDataset)}")
    #     print(toxicDataset[1])
    # Instantiate the model you want to train (e.g., distilRB_base)
    model = distilRB_hate(multilabel=True)


    # Train the model
    loss_per_epoch = train_model(model, toxicDataset)

    torch.cuda.empty_cache()

2572.77587890625
2790.0
2572.77587890625
2624.0


<ipython-input-53-0eb77bb2d325>:3: FutureWarning: `torch.cuda.memory_cached` has been renamed to `torch.cuda.memory_reserved`
  print(torch.cuda.memory_cached()/1024**2)


In [62]:
class ToxicCommentsRobertaEvalDataset(Dataset):
    def __init__(self, df, tokenizer, maxLength=512):
        self.df = df
        self.tokenizer = tokenizer
        self.maxLength = maxLength

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.df.iloc[index]['comment_text']
        labels = self.df.iloc[index][['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.astype(int)
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.maxLength,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(labels, dtype=torch.float32),
        }

In [ ]:
df_test = pd.read_csv('test.csv', quotechar='"')
df_labels = pd.read_csv('test_labels.csv')

df_merged = pd.merge(df_test, df_labels, on='id', how='left')
df_filtered = df_merged[df_merged['toxic'] != -1]  # Filter out rows with -1 labels

# Create the evaluation dataset
eval_dataset = ToxicCommentsRobertaEvalDataset(df_filtered, tokenizer)

# Evaluate the model using the DataLoader
metrics, preds, true_labels, cm  = evaluate_model(model, eval_dataset)
print(metrics)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_metrics(df_filtered, metrics, loss_per_epoch, true_labels, preds, cm):    '''
        Visualizes all metrics after training
    '''
    # 1. Distribution of Labels (Updated)
    # --- Melt the DataFrame to create a 'labels' column ---
    df_melted = df_filtered.melt(
        id_vars=['id', 'comment_text'], # Keep these columns as identifiers
        value_vars=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        var_name='labels', # Name of the new column for label types
        value_name='label_value' # Name of the new column for label values (0 or 1)
    )

    # --- Filter out rows where label_value is -1 (these were the ones we filtered before) ---
    df_melted = df_melted[df_melted['label_value'] != -1]

    plt.figure(figsize=(8, 6))
    # --- Count only positive samples (label_value == 1) ---
    sns.countplot(x='labels', data=df_melted[df_melted['label_value'] == 1])
    plt.title('Distribution of Labels in Test Data')
    plt.xlabel('Label (0: Non-Toxic, 1: Toxic)')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.show()

    # 2. Metrics Visualization (Updated)
    # Calculate F1 scores for each label
    label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']  # Label names
    f1_scores = sklearn.metrics.f1_score(true_labels, preds, average=None) # Calculate F1 for each label

    plt.figure(figsize=(10, 6))
    sns.barplot(x=label_names, y=f1_scores)
    plt.title('F1 Score per Label')
    plt.ylim(0, 1)
    plt.ylabel('F1 Score')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
    plt.show()


    # 3. Confusion Matrix Visualization (Updated for Focus on Positive Cases with Correct Labels)
    label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # --- Plot Confusion Matrix for Positive Cases ---
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for i, label in enumerate(label_names):
        # Extract TP and FP from the confusion matrix
        TP = cm[i, 1, 1]  # True Positives
        FP = cm[i, 0, 1]  # False Positives

        # Create a custom confusion matrix with only TP and FP
        custom_cm = np.array([[FP, TP]]) # Correct custom_cm shape

        sns.heatmap(custom_cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Predicted Negative', 'Predicted Positive'], # Axis labels remain same
                    yticklabels=['Actual Positive'], # update y axis label
                    ax=axes[i])
        axes[i].set_title(f'Confusion Matrix for Label: {label} (Positive Cases)')
        axes[i].set_xlabel('Predicted Label')
        axes[i].set_ylabel('True Label')

    plt.tight_layout()
    plt.show()


    # 4.  Loss Curve (if you saved the loss_per_epoch during training)

    plt.figure(figsize=(10, 6))
    for epoch_losses in loss_per_epoch:
        plt.plot(range(len(epoch_losses)), epoch_losses)
    plt.title('Training Loss per Epoch')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
visualize_metrics(df_filtered, metrics, loss_per_epoch, true_labels, preds, cm)  # Pass the overall confusion matrix